In [1]:
import numpy as np
import tensorflow as tf
print(tf.VERSION)

1.8.0


In [2]:
def cnn_model(features,labels,mode):
    input_layer = tf.reshape(features['x'],[-1,28,28,1])
    conv_1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    pool_1 = tf.layers.max_pooling2d(inputs=conv_1,pool_size = [2,2],strides = 2)
    conv_2 = tf.layers.conv2d(inputs=pool_1,filters=64,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    pool_2 = tf.layers.max_pooling2d(inputs=conv_2,pool_size = [2,2],strides = 2)
    flat = tf.reshape(pool_2,[-1,7 * 7 * 64])
    dense = tf.layers.dense(inputs=flat,units=1024,activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense,rate=0.4,training=(mode==tf.estimator.ModeKeys.TRAIN))
    logits = tf.layers.dense(inputs=dropout,units=10)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    predictions = {
        'classes' : tf.argmax(logits,axis=1),
        'probabilities' : tf.nn.softmax(logits)
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    eval_metric_ops = {
        'accuracy' : tf.metrics.accuracy(labels=labels,predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metric_ops)

In [6]:
mnist = tf.contrib.learn.datasets.load_dataset('mnist')
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels,dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels,dtype=np.int32)
#classifier = tf.estimator.Estimator(model_fn=cnn_model)
train_labels.shape


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


(55000,)

In [4]:
#train_input = tf.estimator.inputs.numpy_input_fn(x={'x' : train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
#classifier.train(input_fn=train_input,steps=10000)

In [5]:
eval_input = tf.estimator.inputs.numpy_input_fn(x={'x' : eval_data},y=eval_labels,num_epochs=1,shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input)
print(eval_results)

ValueError: Could not find trained model in model_dir: /tmp/tmpjwlt4tn9.